In [214]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import mols2grid
import rdkit.Chem.MolStandardize 
import openpyxl

In [215]:
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import mols2grid
import rdkit.Chem.MolStandardize 
from rdkit.Chem.MolStandardize import rdMolStandardize

## Final data for docking

In [216]:
df = pd.read_csv('mixed_data(to_work)_for docking.csv', sep = ',')
df

,Canonical SMILES,Charge,name,Permeability coefficient,Accum_class
0,COc1ccc(CC2N(CCC34C2CC(C(C4)[NH3+])O)C)c3c1O,+,4-2,1913.0,High
1,OC12C(CC3(C2)C(c4c(C3CC1)cccc4C)CC[NH3+])=C,+,4-3,1887.0,High
2,O=C1C2(CC3(C1)C(c4c(C3CC2)cccc4C)C[NH3+])C,+,4-3a,1696.0,High
3,OC12C(CC3(C2)C(c4c(C3CC1)cccc4C)C[NH3+])C,+,4-4,1650.0,High
4,O=C1C2(CC3(C1)C(c4c(C3CC2)cccc4C)C[NH3+])CO,+,4-6,1487.0,High
...,...,...,...,...,...
186,COc1ccc(C=C[C@@]2(C3=CC(C(OC)=C2)=O)CC[N+](C)(...,±,2-100,18.0,Low
187,O=C([C@@H](c1ccccc1)[NH3+])N[C@@H]2C(N3[C@H](C...,±,Ampcillin,45.0,Low
188,O=C(C1=C([O-])[C@@](O)(C(C2=C(c3c([C@](C)([C@H...,+ - -,Tetracycline,1759.0,High
189,Fc1cc2c(N(C=C(C2=O)C([O-])=O)C3CC3)cc1N4CC[NH2...,±,Ciprofloxacin,2263.0,High


In [217]:
df[df['name'] == '4-16']

,Canonical SMILES,Charge,name,Permeability coefficient,Accum_class
11,O=C(CO)OC1C2(C(CCC3(C2C(CC3)=O)C(C(C(C)(C1)C[N...,+,4-16,628.0,High


In [218]:
data_maestro = df[['Canonical SMILES', 'name']]
data_maestro.to_csv('data_maestro.csv',index=False )

In [219]:
data_maestro

,Canonical SMILES,name
0,COc1ccc(CC2N(CCC34C2CC(C(C4)[NH3+])O)C)c3c1O,4-2
1,OC12C(CC3(C2)C(c4c(C3CC1)cccc4C)CC[NH3+])=C,4-3
2,O=C1C2(CC3(C1)C(c4c(C3CC2)cccc4C)C[NH3+])C,4-3a
3,OC12C(CC3(C2)C(c4c(C3CC1)cccc4C)C[NH3+])C,4-4
4,O=C1C2(CC3(C1)C(c4c(C3CC2)cccc4C)C[NH3+])CO,4-6
...,...,...
186,COc1ccc(C=C[C@@]2(C3=CC(C(OC)=C2)=O)CC[N+](C)(...,2-100
187,O=C([C@@H](c1ccccc1)[NH3+])N[C@@H]2C(N3[C@H](C...,Ampcillin
188,O=C(C1=C([O-])[C@@](O)(C(C2=C(c3c([C@](C)([C@H...,Tetracycline
189,Fc1cc2c(N(C=C(C2=O)C([O-])=O)C3CC3)cc1N4CC[NH2...,Ciprofloxacin


In [220]:
list_antibiotics_before_docking = data_maestro['name'].tolist()
print('Nuber of compounds before docking: ' ,len(list_antibiotics_before_docking))

Nuber of compounds before docking:  191


## All antibiotic

In [221]:
name_antibiotic = 'SCE-20'
if name_antibiotic in list_antibiotics:
    print('This antibiotic is already exists in dataset')
    l = df[df['name'] == name_antibiotic]
else:
    print('This antibiotic does not exist in dataset')

This antibiotic does not exist in dataset


## Docking table

In [222]:
#reading initial data from maestro
dff = pd.read_csv('glide-dock_Nature_paper.csv', sep = ',')

In [223]:
#creating new data to work with
dff_new = dff.rename(columns={'Title': 'name'})

#cleaning empty columns 

columns = dff_new.columns
for x in columns:
    number_null = dff_new[x].isnull().sum()
    if number_null == dff_new.shape[0]:
        dff_new.drop(columns = [x], axis = 1, inplace=True)


#cleaning unnecesary columns 
dff_new.drop(columns = ['Entry ID', 'Entry Name', 'Date Added', 'Date Modified',
       'Source Path', 'Source File', 'Source File Index', 'Job Name', 'glide gridfile', 
       'source file', 'Force Field', 'flags', 'Chiralities Consistent','source file index', 
       'mmshare version', 'Variant', 'glide metal', 'glide eff state penalty', 'Energy', 'glide lignum', 'glide confnum', 'glide posenum'], axis = 1, inplace=True)

In [224]:
dff_new.head(7)

,name,glide rotatable bonds,docking score,glide ligand efficiency,glide ligand efficiency sa,glide ligand efficiency ln,glide gscore,glide lipo,glide hbond,glide rewards,...,res:A42 vdw,res:A42 coul,res:A42 hbond,res:A42 dist,res:A42 Eint,res:A16 vdw,res:A16 coul,res:A16 hbond,res:A16 dist,res:A16 Eint
0,Ampcillin,5,-6.747,-0.281,-0.811,-1.615,-6.747,0.000,-0.996,-1.027,...,0.721,-22.104,-0.138,1.877,-21.521,-0.570,-31.409,-0.200,1.947,-32.179
1,4-43,10,-6.710,-0.268,-0.785,-1.591,-6.710,-0.203,-0.826,-1.293,...,-1.105,16.862,0.000,2.891,15.757,-0.096,15.874,0.000,6.169,15.778
2,4-43,10,-6.568,-0.263,-0.768,-1.557,-6.568,-0.333,-0.850,-1.201,...,-1.113,17.215,0.000,2.930,16.103,-0.099,16.013,0.000,5.940,15.914
3,4-43,10,-6.469,-0.259,-0.757,-1.533,-6.469,-0.198,-0.594,-1.293,...,-1.001,15.209,0.000,2.902,14.208,-0.092,14.718,0.000,6.230,14.626
4,4-45,8,-6.378,-0.277,-0.789,-1.542,-6.378,0.000,-0.635,-1.379,...,-1.365,15.126,0.000,2.786,13.761,-0.567,23.327,0.000,3.080,22.760
5,4-45,8,-6.340,-0.276,-0.784,-1.533,-6.340,0.000,-1.136,-1.189,...,-1.419,12.689,0.000,2.409,11.270,-0.768,4.765,-0.137,2.132,3.860
6,4-28a,10,-6.330,-0.275,-0.783,-1.531,-6.330,-0.111,-0.735,-1.169,...,-0.522,34.976,0.000,3.701,34.454,-1.259,39.446,-0.015,2.421,38.172


In [225]:
print('Number of docked compounds: ' ,len(dff_new['name'].unique().tolist()))
print('Number of compounds in dataset: ' ,len(list_antibiotics_before_docking))

Number of docked compounds:  188
Number of compounds in dataset:  191


In [226]:
#ligand's numbers 

list_ligands = []
for x in dff_new.index.tolist():
    y = x+1
    list_ligands.append(y)
list_ligands

dff_new.insert(1, "Ligand", list_ligands)

In [227]:
list_antibiotics = dff_new['name'].unique().tolist()
#list_antibiotics

## Interaction Fingerprints

The following interactions were analyzed: hydrogen bonds (HAccep, HDonor and Ar-Hbond), halogen bonds (XBond), salt-bridge interactions (Salt), π-cation interactions (PiCat), π-π interactions (PiFace, PiEdge) and hydrophobic interactions (HPhob)

In [228]:
int_table = pd.read_csv('glide-dock_Nature_paper_pv_interactions.csv', sep = ',')

In [229]:
int_table

,Type,Ligand,Title,LigResidue,LigAtom,RecResidue,RecAtom,Dist
0,HAccep nn,1,Ampcillin,_:900(UNK),1( ),A:132(ARG),3544(HH12),1.960
1,HAccep cn,1,Ampcillin,_:900(UNK),23( ),A:16(LYS),2753( HZ2),1.947
2,HAccep nn,1,Ampcillin,_:900(UNK),23( ),A:42(ARG),2922(HH12),1.877
3,HAccep nn,1,Ampcillin,_:900(UNK),24( ),A:82(ARG),3197(HH12),1.880
4,HAccep cn,1,Ampcillin,_:900(UNK),24( ),A:82(ARG),3199(HH22),2.121
...,...,...,...,...,...,...,...,...
18692,HPhob,2623,2-62,_:900(UNK),13( ),A:42(ARG),317( CG ),3.990
18693,HPhob,2625,2-62,_:900(UNK),12( ),A:310(TYR),2395( CE2),3.553
18694,HPhob,2625,2-62,_:900(UNK),13( ),A:310(TYR),2395( CE2),3.766
18695,HPhob,2626,2-62,_:900(UNK),13( ),A:14(TYR),110( CD1),3.424


In [230]:
interactions = int_table.rename(columns={'Title': 'name'})

interactions.drop(columns = ['LigResidue','LigAtom', 'RecAtom'], axis = 1, inplace=True)
interactions

,Type,Ligand,name,RecResidue,Dist
0,HAccep nn,1,Ampcillin,A:132(ARG),1.960
1,HAccep cn,1,Ampcillin,A:16(LYS),1.947
2,HAccep nn,1,Ampcillin,A:42(ARG),1.877
3,HAccep nn,1,Ampcillin,A:82(ARG),1.880
4,HAccep cn,1,Ampcillin,A:82(ARG),2.121
...,...,...,...,...,...
18692,HPhob,2623,2-62,A:42(ARG),3.990
18693,HPhob,2625,2-62,A:310(TYR),3.553
18694,HPhob,2625,2-62,A:310(TYR),3.766
18695,HPhob,2626,2-62,A:14(TYR),3.424


In [231]:
# number of found residues after docking

jkj = interactions['RecResidue'].unique().tolist()
len(jkj)

52

In [232]:
#types of found interactions

binds = interactions['Type'].unique().tolist()
print(len(binds))
binds


14


['HAccep nn',
 'HAccep cn',
 'HDonor nc',
 'HDonor cn',
 'HDonor nn',
 'HDonor cc',
 'HAccep cc',
 'HAccep nc',
 'XBond',
 'Salt',
 'PiCat',
 'PiFace',
 'PiEdge',
 'HPhob']

In [233]:
table_int = pd.pivot_table(interactions, values='Type', index=['name', 'Ligand'], columns=['RecResidue'], aggfunc = np.sum, fill_value=0)

In [234]:
interaction_residues = table_int.columns
interaction_residues

Index(['A:1(ALA)', 'A:102(TYR)', 'A:106(TYR)', 'A:107(ASP)', 'A:110(GLY)',
       'A:111(TYR)', 'A:113(ASP)', 'A:114(MET)', 'A:115(LEU)', 'A:116(PRO)',
       'A:117(GLU)', 'A:118(PHE)', 'A:119(GLY)', 'A:12(ASP)', 'A:120(GLY)',
       'A:121(ASP)', 'A:123(ALA)', 'A:124(TYR)', 'A:125(SER)', 'A:132(ARG)',
       'A:14(TYR)', 'A:15(GLY)', 'A:16(LYS)', 'A:167(ARG)', 'A:168(ARG)',
       'A:18(VAL)', 'A:20(LEU)', 'A:22(TYR)', 'A:226(TYR)', 'A:262(GLN)',
       'A:300(THR)', 'A:302(TYR)', 'A:305(LYS)', 'A:308(SER)', 'A:310(TYR)',
       'A:32(TYR)', 'A:33(GLY)', 'A:337(VAL)', 'A:339(GLN)', 'A:340(PHE)',
       'A:38(MET)', 'A:40(TYR)', 'A:42(ARG)', 'A:46(LYS)', 'A:60(GLN)',
       'A:62(GLU)', 'A:64(ASN)', 'A:66(GLN)', 'A:80(LYS)', 'A:82(ARG)',
       'A:83(LEU)', 'A:85(PHE)'],
      dtype='object', name='RecResidue')

In [235]:
#interaction fingerpronts for each residue

for x in range(table_int.shape[0]):
    for y in range(table_int.shape[1]):
        cell = str(table_int.iloc[x, y])
        #print(cell)
        list_int = []
        for bond in binds:
            if bond in cell:
                #print('YES', bond)
                list_int.append("1")
            else:
                list_int.append("0")
        #print(list_int)
        table_int.iloc[x,y] =  " ".join(list_int)
        #table_int.iloc[x,y] = list_int

In [236]:
#format column names
table_int.columns = [''.join(str(s).strip() for s in col if s) for col in table_int.columns]

#reset index
table_int.reset_index(inplace=True)

In [237]:
table_int

,name,Ligand,A:1(ALA),A:102(TYR),A:106(TYR),A:107(ASP),A:110(GLY),A:111(TYR),A:113(ASP),A:114(MET),...,A:42(ARG),A:46(LYS),A:60(GLN),A:62(GLU),A:64(ASN),A:66(GLN),A:80(LYS),A:82(ARG),A:83(LEU),A:85(PHE)
0,2-1,133,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 1 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
1,2-1,226,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 1 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
2,2-1,239,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 1 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
3,2-1,267,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 1 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 1 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
4,2-1,270,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 1 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 0 0 0 0 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,Tetracycline,895,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 1 0 0 0 0 0 0 0 1 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 1 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 1 0 0 1 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
2610,Tetracycline,938,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,1 1 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 1 0 1 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
2611,Tetracycline,1043,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 

In [238]:
# interaction_1Dstring

residues = table_int.columns.tolist()

table_int['interaction_1Dstring'] = 0

for x in range(table_int.shape[0]):
    oneD_int = []
    for y in residues[2:]:
        cell_1 = table_int.loc[x,y]
        oneD_int.append(cell_1)
        
    table_int.loc[x, 'interaction_1Dstring'] = " ".join(oneD_int)
    #table_int.loc[x, 'interaction_1Dstring'] = oneD_int

In [193]:
# number of bits, number of residues * types of interactions

len(table_int.loc[2456, 'interaction_1Dstring'].split(' '))

728

In [239]:
FINAL = pd.merge(dff_new, table_int,  how='left', left_on=['name','Ligand'], right_on = ['name','Ligand'])

In [246]:
FINAL.head()

,name,Ligand,glide rotatable bonds,docking score,glide ligand efficiency,glide ligand efficiency sa,glide ligand efficiency ln,glide gscore,glide lipo,glide hbond,...,A:46(LYS),A:60(GLN),A:62(GLU),A:64(ASN),A:66(GLN),A:80(LYS),A:82(ARG),A:83(LEU),A:85(PHE),interaction_1Dstring
0,Ampcillin,1,5,-6.747,-0.281,-0.811,-1.615,-6.747,0.000,-0.996,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 1 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,4-43,2,10,-6.710,-0.268,-0.785,-1.591,-6.710,-0.203,-0.826,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,4-43,3,10,-6.568,-0.263,-0.768,-1.557,-6.568,-0.333,-0.850,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,4-43,4,10,-6.469,-0.259,-0.757,-1.533,-6.469,-0.198,-0.594,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,4-45,5,8,-6.378,-0.277,-0.789,-1.542,-6.378,0.000,-0.635,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


# Data for machine learning course

In [255]:
data_course = FINAL.copy()
data_course.head()

,name,Ligand,glide rotatable bonds,docking score,glide ligand efficiency,glide ligand efficiency sa,glide ligand efficiency ln,glide gscore,glide lipo,glide hbond,...,A:46(LYS),A:60(GLN),A:62(GLU),A:64(ASN),A:66(GLN),A:80(LYS),A:82(ARG),A:83(LEU),A:85(PHE),interaction_1Dstring
0,Ampcillin,1,5,-6.747,-0.281,-0.811,-1.615,-6.747,0.000,-0.996,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 1 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,4-43,2,10,-6.710,-0.268,-0.785,-1.591,-6.710,-0.203,-0.826,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,4-43,3,10,-6.568,-0.263,-0.768,-1.557,-6.568,-0.333,-0.850,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,4-43,4,10,-6.469,-0.259,-0.757,-1.533,-6.469,-0.198,-0.594,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,4-45,5,8,-6.378,-0.277,-0.789,-1.542,-6.378,0.000,-0.635,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [256]:
#cleaning unnecesary columns 
data_course.drop(columns = ['glide rotatable bonds',
 'docking score',
 'glide ligand efficiency',
 'glide ligand efficiency sa',
 'glide ligand efficiency ln',
 'glide gscore',
 'glide lipo',
 'glide hbond',
 'glide rewards',
 'glide evdw',
 'glide ecoul',
 'glide erotb',
 'glide esite',
 'glide emodel',
 'glide energy',
 'glide einternal',
 'res:A132 vdw',
 'res:A132 coul',
 'res:A132 hbond',
 'res:A132 dist',
 'res:A132 Eint',
 'res:A117 vdw',
 'res:A117 coul',
 'res:A117 hbond',
 'res:A117 dist',
 'res:A117 Eint',
 'res:A113 vdw',
 'res:A113 coul',
 'res:A113 hbond',
 'res:A113 dist',
 'res:A113 Eint',
 'res:A82 vdw',
 'res:A82 coul',
 'res:A82 hbond',
 'res:A82 dist',
 'res:A82 Eint',
 'res:A42 vdw',
 'res:A42 coul',
 'res:A42 hbond',
 'res:A42 dist',
 'res:A42 Eint',
 'res:A16 vdw',
 'res:A16 coul',
 'res:A16 hbond',
 'res:A16 dist',
 'res:A16 Eint', 'interaction_1Dstring'], axis = 1, inplace=True)

In [293]:
data_course.shape

(2626, 54)

In [300]:
dict_new = {}

Canonical_SMILES = []
chargee = []
coef = []
classs = []
#namee = []


for indd in data_course.index:
    name1 = data_course.loc[indd, 'name']
    exp_values = df[df['name'] == name1]

    indd_2 = list(exp_values.index)
    #namee.append(name1)
    Canonical_SMILES.append(exp_values.loc[indd_2[0], 'Canonical SMILES'])
    chargee.append(exp_values.loc[indd_2[0], 'Charge'])
    coef.append(exp_values.loc[indd_2[0], 'Permeability coefficient'])
    classs.append(exp_values.loc[indd_2[0], 'Accum_class'])

dict_new['Canonical SMILES'] = Canonical_SMILES
#dict_new['name'] = namee
dict_new['Permeability coefficient'] = coef
dict_new['Accumulation_class'] = classs
dict_new['Charge'] = chargee

data_numb = pd.DataFrame(dict_new)
print(data_numb.shape)
data_numb.head()


(2626, 4)


,Canonical SMILES,Permeability coefficient,Accumulation_class,Charge
0,O=C([C@@H](c1ccccc1)[NH3+])N[C@@H]2C(N3[C@H](C...,45.0,Low,±
1,O=C(C(C(C)C)[NH3+])OCC(CO)OCn1cnc2c1N=C(NC2=O)N,165.0,Low,+
2,O=C(C(C(C)C)[NH3+])OCC(CO)OCn1cnc2c1N=C(NC2=O)N,165.0,Low,+
3,O=C(C(C(C)C)[NH3+])OCC(CO)OCn1cnc2c1N=C(NC2=O)N,165.0,Low,+
4,O=C(C(C(C)C)[NH3+])OCCOCn1cnc2c1N=C(NC2=O)N,141.0,Low,+


In [301]:
result = pd.concat([data_numb, data_course], axis=1)
result

,Canonical SMILES,Permeability coefficient,Accumulation_class,Charge,name,Ligand,A:1(ALA),A:102(TYR),A:106(TYR),A:107(ASP),...,A:42(ARG),A:46(LYS),A:60(GLN),A:62(GLU),A:64(ASN),A:66(GLN),A:80(LYS),A:82(ARG),A:83(LEU),A:85(PHE)
0,O=C([C@@H](c1ccccc1)[NH3+])N[C@@H]2C(N3[C@H](C...,45.0,Low,±,Ampcillin,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 1 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
1,O=C(C(C(C)C)[NH3+])OCC(CO)OCn1cnc2c1N=C(NC2=O)N,165.0,Low,+,4-43,2,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
2,O=C(C(C(C)C)[NH3+])OCC(CO)OCn1cnc2c1N=C(NC2=O)N,165.0,Low,+,4-43,3,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
3,O=C(C(C(C)C)[NH3+])OCC(CO)OCn1cnc2c1N=C(NC2=O)N,165.0,Low,+,4-43,4,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
4,O=C(C(C(C)C)[NH3+])OCCOCn1cnc2c1N=C(NC2=O)N,141.0,Low,+,4-45,5,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621,O(CC=C)[C@H]1[C@H]2OC[C@@H](OCC=C)[C@H]2OC1,139.0,Low,neutral,2-62,2622,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
2622,O(CC=C)[C@H]1[C@H]2OC[C@@H](OCC=C)[C@H]2OC1,139.0,Low,neutral,2-62,2623,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
2623,O(CC=C)[C@H]1[C@H]2OC[C@@H](OCC=C)[C@H]2OC1,139.0,Low,neutral,2-62,2624,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2624,O(CC=C)[C@H]1[C@H]2OC[C@@H](OCC=C)[C@H]2OC1,139.0,Low,neutral,2-62,2625,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 

In [302]:
result.to_csv(r'C:\Users\olshana1\Documents\new-progect-about-gram-negative-bacteria-main\work with NATURE dataset\Gram Negative Permeation - OmpF- applying for Nature_data/data_final_project.csv', index=False, header=True)

In [240]:
for_nan = FINAL.loc[0, 'A:46(LYS)']
true_false = FINAL.isnull()

In [196]:
for sample in true_false.index:
    for feature_1 in true_false.columns.tolist():
        if true_false.loc[sample, feature_1] == True:
            #print(sample, feature_1)
            FINAL.loc[sample, feature_1] = for_nan

In [197]:
FINAL.head()

,name,Ligand,glide rotatable bonds,docking score,glide ligand efficiency,glide ligand efficiency sa,glide ligand efficiency ln,glide gscore,glide lipo,glide hbond,...,A:46(LYS),A:60(GLN),A:62(GLU),A:64(ASN),A:66(GLN),A:80(LYS),A:82(ARG),A:83(LEU),A:85(PHE),interaction_1Dstring
0,Ampcillin,1,5,-6.747,-0.281,-0.811,-1.615,-6.747,0.000,-0.996,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 1 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,4-43,2,10,-6.710,-0.268,-0.785,-1.591,-6.710,-0.203,-0.826,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,4-43,3,10,-6.568,-0.263,-0.768,-1.557,-6.568,-0.333,-0.850,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,4-43,4,10,-6.469,-0.259,-0.757,-1.533,-6.469,-0.198,-0.594,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,4-45,5,8,-6.378,-0.277,-0.789,-1.542,-6.378,0.000,-0.635,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [198]:
FINAL.isnull().sum()

name                       0
Ligand                     0
glide rotatable bonds      0
docking score              0
glide ligand efficiency    0
                          ..
A:80(LYS)                  0
A:82(ARG)                  0
A:83(LEU)                  0
A:85(PHE)                  0
interaction_1Dstring       0
Length: 101, dtype: int64

In [208]:
FINAL.shape

(2626, 101)

In [199]:
FINAL.to_csv(r'C:\Users\olshana1\Documents\new-progect-about-gram-negative-bacteria-main\work with NATURE dataset\Gram Negative Permeation - OmpF- applying for Nature_data/FINAL.csv', index=False, header=True)

# Taking a pose based on interaction fingerprints 

## Voting based on majority of the interaction existence
#### FINAL_pose1 dataframe

In [200]:
#new dataframe 

new_col_t = table_int.columns.tolist()
new_col_t.remove('Ligand')
new_col_t.remove('interaction_1Dstring')

FINAL_pose1 = pd.DataFrame(columns=new_col_t)
FINAL_pose1['name'] = list_antibiotics


In [201]:
for antibiotic in list_antibiotics:
    for residue in new_col_t[1:]:

        #take antibiotic 
        test_ant = FINAL.loc[FINAL['name'] == antibiotic, [residue]]

        #take residue
        test_dat = pd.DataFrame(columns=list(range(1, len(binds)+1)))
        for bit_str in test_ant[residue]:
            test_dat.loc[len(test_dat)] = bit_str.split(' ')

        #voting
        ones_ziros = []
        for col in range(1, len(binds)+1):
            ones = 0
            for bit in test_dat.loc[:, col]:
                if bit == '1':
                    ones = ones + 1
            ones_ziros.append(ones)

        #generating new string 
        cut_off = test_dat.shape[0]/2

        list_bits_new = []
        for sum in ones_ziros:
            if sum > cut_off:
                list_bits_new.append("1")
            else:
                list_bits_new.append("0")
        
        gfg = " ".join(list_bits_new)

        FINAL_pose1.loc[FINAL_pose1['name'] == antibiotic, [residue]] = gfg
        #print(antibiotic, residue, gfg)


In [202]:
FINAL_pose1

,name,A:1(ALA),A:102(TYR),A:106(TYR),A:107(ASP),A:110(GLY),A:111(TYR),A:113(ASP),A:114(MET),A:115(LEU),...,A:42(ARG),A:46(LYS),A:60(GLN),A:62(GLU),A:64(ASN),A:66(GLN),A:80(LYS),A:82(ARG),A:83(LEU),A:85(PHE)
0,Ampcillin,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 1 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
1,4-43,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 1 1 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
2,4-45,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
3,4-28a,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 1 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
4,4-55a,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 1 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,4-29,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0
184,3-52,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,...,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0

## Voting based on at least ONE interaction existence
#### FINAL_pose3 dataframe

In [203]:
FINAL_pose3 = pd.DataFrame(columns=new_col_t)
FINAL_pose3['name'] = list_antibiotics

In [204]:
for antibiotic in list_antibiotics:
    for residue in new_col_t[1:]:

        #take antibiotic 
        test_ant = FINAL.loc[FINAL['name'] == antibiotic, [residue]]

        #take residue
        test_dat = pd.DataFrame(columns=list(range(1, len(binds)+1)))
        for bit_str in test_ant[residue]:
            test_dat.loc[len(test_dat)] = bit_str.split(' ')

        #voting
        ones_ziros = []
        for col in range(1, len(binds)+1):
            ones = 0
            for bit in test_dat.loc[:, col]:
                if bit == '1':
                    ones = ones + 1
            ones_ziros.append(ones)

        list_bits_new = []
        for sum in ones_ziros:
            if sum > 0:
                list_bits_new.append("1")
            else:
                list_bits_new.append("0")
        
        gfg = " ".join(list_bits_new)

        FINAL_pose3.loc[FINAL_pose3['name'] == antibiotic, [residue]] = gfg
        #print(antibiotic, residue, gfg)


## Taking the low energy pose for each lingand
#### FINAL_pose2 dataframe

In [205]:
#empty DataFrame
FINAL_pose2 = pd.DataFrame(columns = FINAL.columns)

#adding pose with low energy
for x in list_antibiotics:
    x = FINAL[FINAL['name'] == x]
    min_glide_gscore= x['glide gscore'].min()
    l = x[x['glide gscore'] == min_glide_gscore]
    FINAL_pose2 = pd.concat([FINAL_pose2, l])

## Charge, Permeability coefficient, ASP113 for all experiments

In [206]:
list_experiments = [FINAL_pose1, FINAL_pose2, FINAL_pose3]
important_residues = ['A:113(ASP)', 'A:117(GLU)', 'A:16(LYS)', 'A:42(ARG)', 'A:82(ARG)', 'A:132(ARG)']

for experiment in list_experiments:

    list_charge = []
    list_Permeability_coefficient = []
    list_Canonical_SMILES = []

    for x in experiment['name']:
        string = df[df['name'] == x]
        charge = string['Charge'].values
        Permeability_coefficient = int(string['Permeability coefficient'])
        Canonical_SMILES = string['Canonical SMILES'].values
        
        list_charge.append(charge[0])
        list_Permeability_coefficient.append(Permeability_coefficient)
        list_Canonical_SMILES.append(Canonical_SMILES[0])

    experiment.insert(1, "Permeability coefficient", list_Permeability_coefficient, False)
    experiment.insert(2, "Charge", list_charge, False)
    experiment.insert(experiment.shape[1], "Canonical SMILES" , list_Canonical_SMILES, False)

    #all important bonds

    for residue in important_residues:
        bond = '_bond'
        experiment[residue + bond] = 0
        for p in experiment.index:
            string = experiment.loc[p, residue]
            a113_number = 0
            for y in string:
                if y == '1':
                    a113_number = a113_number + 1
                else:
                    a113_number = a113_number + 0
        
            if  a113_number > 0:
                experiment.loc[p, residue + bond] = "1"
            else:
                experiment.loc[p, residue + bond] = "0"

C:\Users\olshana1\AppData\Local\Temp\ipykernel_19052\2013265626.py:13: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  Permeability_coefficient = int(string['Permeability coefficient'])
C:\Users\olshana1\AppData\Local\Temp\ipykernel_19052\2013265626.py:13: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  Permeability_coefficient = int(string['Permeability coefficient'])
C:\Users\olshana1\AppData\Local\Temp\ipykernel_19052\2013265626.py:13: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  Permeability_coefficient = int(string['Permeability coefficient'])


In [207]:
FINAL_pose1.to_csv(r'C:\Users\olshana1\Documents\new-progect-about-gram-negative-bacteria-main\work with NATURE dataset\Gram Negative Permeation - OmpF- applying for Nature_data/FINAL_pose1.csv', index=False, header=True)
FINAL_pose2.to_csv(r'C:\Users\olshana1\Documents\new-progect-about-gram-negative-bacteria-main\work with NATURE dataset\Gram Negative Permeation - OmpF- applying for Nature_data/FINAL_pose2.csv', index=False, header=True)
FINAL_pose3.to_csv(r'C:\Users\olshana1\Documents\new-progect-about-gram-negative-bacteria-main\work with NATURE dataset\Gram Negative Permeation - OmpF- applying for Nature_data/FINAL_pose3.csv', index=False, header=True)

#### FINAL - dataframe with all poses
#### FINAL_pose1 - Taking a pose based on interaction fingerprints (based on the majority)
#### FINAL_pose2 - Taking the low energy pose for each lingand
#### FINAL_pose3 dataframe - Voting based on at least ONE interaction existence